# 更改已提交的健康卡记录

## 故事

前几天突发奇想健康卡可以提前提交好几天的，于是乎，我在一天内把未来7天的健康卡都提交完了。当时是下午2点，第二天我一看发现不对，辅导员让我们每天中午12点前提交，如果我的提交记录是下午2点的，有点不太完美。于是我又构造了一个早上8点的健康卡提交记录，想覆盖到数据库里面去。可是提交完之后发现没用，才想到这个破系统只会显示最晚一次的健康卡提交记录，相当于我只有提交比下午2点还晚的记录才可以。

之后我就没去管它，直到今天辅导员来找我麻烦了...

他说看到我每天都有两条提交记录，要让我仅保留一条，免得搞乱他数据。

In [1]:
import requests
import json

In [12]:
# 写文件的辅助函数
def output_to_file(filename, data):
    with open(filename,'w') as fd:
        fd.write(json.dumps(data,indent=4,ensure_ascii=False))

## 身份认证

In [6]:
JSESSIONID = "4567E39DACFBC754AB50C69AD80C0BE0"
headers = {
    "x-requested-with": "XMLHttpRequest",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) ,Chrome/84.0.4147.105 Safari/537.36",
    "content-type": "text/json",
    "origin": "https://work.jluzh.edu.cn",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "cors",
    "sec-fetch-dest": "empty",
    "referer": "https://work.jluzh.edu.cn/default/work/jlzh/jkxxtb/jkxxcj.jsp",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-length": "896",
    "cookie": "JSESSIONID=" + JSESSIONID
}

## 覆盖记录函数

修改自动提交函数，saveOrUpdate 本来是用于提交健康卡的，但是它也有 Update 功能，上网查一下发现只要 entity 里面有 id 字段就可以 update 指定 id 的记录信息。 

函数的输入参数分别是：用于覆盖的信息、需要覆盖的记录 ID、用于身份认证的请求头

In [14]:
def cover(coverDataDic, coverId, headers):
    coverDataDic["entity"]["id"] = coverId
    coverData = json.dumps(coverDataDic,ensure_ascii=False)
    url = "https://work.jluzh.edu.cn/default/work/jlzh/jkxxtb/com.sudytech.portalone.base.db.saveOrUpdate.biz.ext"
    r = requests.post(url,headers = headers,data=coverData.encode("utf-8"))
    output_to_file("data/cover_temp_output.json",r.json())
    print("覆盖成功，返回数据已输出至 data/cover_temp_output.json")

## 构造用于覆盖的健康卡

因为无法删除，所以我的解决办法是把要移除的多余健康卡移动到之前没填过的日期，这样辅导员在查当天的时候就不会看到我每天多条记录了。

In [21]:
from auto_health import query_record
last_card = query_record(20190105, headers)[0]
health_card_data = {
    "entity": {
        "sqrid": last_card["SQRID"],
        "sqbmid": last_card["SQBMID"],
        "fdygh": last_card["FDYGH"],
        "rysf": last_card["RYSF"],
        "bt": "2021-01-05 曾亦凡 健康卡填报",
        "sqrmc": last_card["SQRMC"],
        "gh": last_card["GH"],
        "xb": last_card["XB"],
        "sqbmmc": last_card["SQBMMC"],
        "nj": last_card["NJ"],
        "zymc": last_card["ZYMC"],
        "bjmc": last_card["BJMC"],
        "fdymc": last_card["FDYMC"],
        "ssh": last_card["SSH"],
        "lxdh": last_card["LXDH"],
        "tbrq": "2021-01-05",
        "tjsj": "2021-01-05 08:30",
        "xjzdz": last_card["XJZDZ"],
        "jqqx": last_card["JQQX"],
        "sfqwhb": last_card["SFQWHB"],
        "sfjchbjry": last_card["SFJCHBJRY"],
        "sfjwhy": last_card["SFJWHY"],
        "sfjwhygjdq": last_card["SFJWHYGJDQ"],
        "xrywz": last_card["XRYWZ"],
        "jtdz": last_card["JTDZ"],
        "grjkzk": last_card["GRJKZK"],
        "jrtw": last_card["JRTW"],
        "qsjkzk": last_card["QSJKZK"],
        "jkqk": last_card["JKQK"],
        "cn": [
            "本人承诺登记后、到校前不再前往其他地区"
        ],
        "bz": last_card["BZ"],
        "_ext": "{}",
        "__type": "sdo:com.sudytech.work.jlzh.jkxxtb.jkxxcj.TJlzhJkxxtb"
    }
}

查询学号 20190105 成功，共计查询到 1 份记录。 仅查询今日=True


In [22]:
cover(health_card_data,2152208,headers)

覆盖成功，返回数据已输出至 data/cover_temp_output.json
